## Imports and google drive stuff

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/'2025 Uni'/IR2

/content/drive/MyDrive/2025 Uni/IR2


In [ ]:

pip install evaluate

In [ ]:
pip install xformers

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, EarlyStoppingCallback, get_scheduler, AutoConfig
import torch.nn as nn
from evaluate import load
from torch.utils.data import DataLoader
import torch
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score


## Code

In [ ]:
def initialize_dataset(name):
    dataset = load_dataset(name)

    # Some preprocessing if needed
    return dataset

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512) # TODO: correct length


In [ ]:
dataset_name = "SetFit/sst5"
dataset = initialize_dataset(dataset_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
# Initialize the BERT tokenizer
# model_name = "chandar-lab/NeoBERT"
# model_name = "google-bert/bert-base-uncased"
# model_name = "FacebookAI/roberta-base"
# model_name = "answerdotai/ModernBERT-base"
model_name = "microsoft/deberta-v3-large"


tokenizer_name = model_name
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

# Apply the tokenizer to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

model = AutoModelForSequenceClassification.from_pretrained(model_name, trust_remote_code=True, num_labels=5)


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/2210 [00:00<?, ? examples/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# if model_name == "microsoft/deberta-v3-large":


training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    lr_scheduler_type="linear",
    warmup_ratio=0.1,
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=100,
    fp16=True,
    report_to="none"
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Define a custom compute_metrics function\
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {
        "f1_macro": f1_score(labels, predictions, average="macro"),
        "f1_weighted": f1_score(labels, predictions, average="weighted"),
        "accuracy": accuracy_score(labels, predictions),
        "precision": recall_score(labels, predictions, average="weighted"),
        "recall": precision_score(labels, predictions, average="weighted")
    }

trainer = Trainer(
    model=model,                        # Pre-trained BERT model
    args=training_args,                 # Training arguments
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,        # Efficient batching
    compute_metrics=compute_metrics     # Custom metric
)

trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))


# has a lot of good code:
# https://medium.com/@heyamit10/fine-tuning-bert-for-classification-a-practical-guide-b8c1c56f252c
# Look at hyperparam tuning (blog uses optima)


/tmp/ipython-input-1988612963.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Start training
trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 66.12 MiB is free. Process 78302 has 14.67 GiB memory in use. Of the allocated memory 14.54 GiB is allocated by PyTorch, and 7.36 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
trainer.save_model("./{}_classification_3E".format(model_name))

In [ ]:
results = trainer.evaluate()
print(results)

# Evaluation code

In [ ]:
# Define a custom compute_metrics function\
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {
        "f1_macro": f1_score(labels, predictions, average="macro"),
        "f1_weighted": f1_score(labels, predictions, average="weighted"),
        "accuracy": accuracy_score(labels, predictions),
        "precision": recall_score(labels, predictions, average="weighted"),
        "recall": precision_score(labels, predictions, average="weighted")
    }

def get_test_results(model_name):
  dataset_name = "SetFit/sst5"
  dataset = initialize_dataset(dataset_name)

  tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

  def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512) # TODO: correct length

  # Apply the tokenizer to the dataset
  tokenized_datasets = dataset.map(tokenize_function, batched=True)

  model_location = "./{}_classification_3E".format(model_name)
  model = AutoModelForSequenceClassification.from_pretrained(model_location, trust_remote_code=True, num_labels=5)

  training_args = TrainingArguments(
      output_dir="./results",
      per_device_eval_batch_size=16,
      report_to="none"
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      tokenizer=tokenizer,
      compute_metrics=compute_metrics
  )

  results = trainer.evaluate(tokenized_datasets['test'])
  print("Test results for model {}".format(model_name))
  print(results)


In [ ]:
# BERT Results
get_test_results("google-bert/bert-base-uncased")

Repo card metadata block was not found. Setting CardData to empty.


Map:   0%|          | 0/8544 [00:00<?, ? examples/s]

Map:   0%|          | 0/1101 [00:00<?, ? examples/s]

Map:   0%|          | 0/2210 [00:00<?, ? examples/s]

/tmp/ipython-input-3125825141.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Test results for model google-bert/bert-base-uncased
{'eval_loss': 1.1327016353607178, 'eval_model_preparation_time': 0.0048, 'eval_f1_macro': 0.4932114215868113, 'eval_f1_weighted': 0.49785117834300086, 'eval_accuracy': 0.502262443438914, 'eval_precision': 0.502262443438914, 'eval_recall': 0.5135151036751566, 'eval_runtime': 79.017, 'eval_samples_per_second': 27.969, 'eval_steps_per_second': 1.759}


In [ ]:
# RoBERTa Results
get_test_results("FacebookAI/roberta-base")

Repo card metadata block was not found. Setting CardData to empty.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Map:   0%|          | 0/8544 [00:00<?, ? examples/s]

Map:   0%|          | 0/1101 [00:00<?, ? examples/s]

Map:   0%|          | 0/2210 [00:00<?, ? examples/s]

/tmp/ipython-input-3125825141.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Test results for model FacebookAI/roberta-base
{'eval_loss': 1.0774277448654175, 'eval_model_preparation_time': 0.0027, 'eval_f1_macro': 0.4789828648779121, 'eval_f1_weighted': 0.4915432860311305, 'eval_accuracy': 0.5190045248868779, 'eval_precision': 0.5190045248868779, 'eval_recall': 0.5076485380398179, 'eval_runtime': 63.2796, 'eval_samples_per_second': 34.924, 'eval_steps_per_second': 2.197}


In [ ]:
# ModernBERT Results
get_test_results("answerdotai/ModernBERT-base")

Repo card metadata block was not found. Setting CardData to empty.


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Map:   0%|          | 0/8544 [00:00<?, ? examples/s]

Map:   0%|          | 0/1101 [00:00<?, ? examples/s]

Map:   0%|          | 0/2210 [00:00<?, ? examples/s]

/tmp/ipython-input-3125825141.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
W1205 12:09:48.330000 217 torch/_inductor/utils.py:1613] [1/0_1] Not enough SMs to use max_autotune_gemm mode


Test results for model answerdotai/ModernBERT-base
{'eval_loss': 0.9988135695457458, 'eval_model_preparation_time': 0.0032, 'eval_f1_macro': 0.5209368537548373, 'eval_f1_weighted': 0.5370026488033363, 'eval_accuracy': 0.5493212669683258, 'eval_precision': 0.5493212669683258, 'eval_recall': 0.5468221403240926, 'eval_runtime': 109.0032, 'eval_samples_per_second': 20.275, 'eval_steps_per_second': 1.275}


In [ ]:
# NeoBERT Results
get_test_results("chandar-lab/NeoBERT")

Repo card metadata block was not found. Setting CardData to empty.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Map:   0%|          | 0/8544 [00:00<?, ? examples/s]

Map:   0%|          | 0/1101 [00:00<?, ? examples/s]

Map:   0%|          | 0/2210 [00:00<?, ? examples/s]

/tmp/ipython-input-3125825141.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Test results for model chandar-lab/NeoBERT
{'eval_loss': 1.0502057075500488, 'eval_model_preparation_time': 0.0028, 'eval_f1_macro': 0.5108024309319426, 'eval_f1_weighted': 0.5166893028844078, 'eval_accuracy': 0.5217194570135747, 'eval_precision': 0.5217194570135747, 'eval_recall': 0.5236163817067985, 'eval_runtime': 173.5273, 'eval_samples_per_second': 12.736, 'eval_steps_per_second': 0.801}
